In [1]:
try:
    import onnx
    import onnxruntime
    import tflite_runtime.interpreter as tflite
    import onnxsim
except:
    !pip install tflite-runtime > /dev/null
    !pip install onnxruntime-gpu > /dev/null
    !pip install onnx-tf > /dev/null
    !pip install onnxsim > /dev/null

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json 

import torch.onnx
print('torch.onnx.producer_version', torch.onnx.producer_version)

import onnx
import onnxruntime
import onnxsim

import tflite_runtime.interpreter as tflite
# kaggle requirement TensorFlow Lite Runtime v2.9.1.

import tflite_runtime
print('tflite_runtime.__version__', tflite_runtime.__version__)

import tensorflow as tf
print('tf.__version__', tf.__version__)

# import onnx2tf
from onnx_tf.backend import prepare

import torch
import torch.nn.functional as F
import torch.nn as nn

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# from all import *
import torch
# from common import *
import tensorflow as tf
import subprocess

import onnx
import onnxruntime
import onnxsim
from onnx_tf.backend import prepare

torch.onnx.producer_version 1.13.0
tflite_runtime.__version__ 2.11.0
tf.__version__ 2.11.0


In [2]:
checkpoint_files = [
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold0.pth',
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold4.pth',
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold8.pth',
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold9.pth',
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold10.pth',
'/kaggle/input/asl-dataset/models_exp81/models_exp81/asl_model_fold18.pth',
]
name = 'exp81'

In [3]:
## Config 
num_landmark = 543
max_length = 256
num_class  = 250
num_point  = 1050 #960 #82  # LIP, LHAND, RHAND

embed_dim = 384 #512
num_block = 1
num_head = 8

convert_dir = 'convert_dir' #f'{fold_dir}/convert/{name}'
os.makedirs(convert_dir, exist_ok=True)

fold = 'all'
input_net_k_tf_file = f'{convert_dir}/input_net.fold_{fold}.k.tf'
input_net_k_tflite_file = f'{convert_dir}/input_net.fold_{fold}.k.tflite'

ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [4]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, embed_dim),
        )
    def forward(self, x):
        return self.mlp(x)
    


def positional_encoding(length, embed_dim):
    dim = embed_dim//2
    position = np.arange(length)[:, np.newaxis]     # (seq, 1)
    dim = np.arange(dim)[np.newaxis, :]/dim   # (1, dim)
    angle = 1 / (10000**dim)         # (1, dim)
    angle = position * angle    # (pos, dim)
    pos_embed = np.concatenate(
        [np.sin(angle), np.cos(angle)],
        axis=-1
    )
    pos_embed = torch.from_numpy(pos_embed).float()
    return pos_embed


### Convert InputNet to tflite

In [5]:

class InputNet(tf.keras.layers.Layer):
    def __init__(self, ):
        super(InputNet, self).__init__()
        self.lip = tf.constant([
            61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
            291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
            95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
        ])
        self.spose = tf.constant([
            504, 502, 500, 501, 503, 505, 512, 513
        ])
        self.triu_index = tf.constant([
            1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
            14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28,
            29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
            45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
            58, 59, 60, 61, 62, 67, 68, 69, 70, 71, 72, 73, 74,
            75, 76, 77, 78, 79, 80, 81, 82, 83, 89, 90, 91, 92,
            93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 111,
            112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
            125, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
            145, 146, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
            166, 167, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
            188, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 221,
            222, 223, 224, 225, 226, 227, 228, 229, 230, 243, 244, 245, 246,
            247, 248, 249, 250, 251, 265, 266, 267, 268, 269, 270, 271, 272,
            287, 288, 289, 290, 291, 292, 293, 309, 310, 311, 312, 313, 314,
            331, 332, 333, 334, 335, 353, 354, 355, 356, 375, 376, 377, 397,
            398, 419,
        ])
        self.lhand = (468, 489)
        self.rhand = (522, 543)
        self.max_length = max_length
        self.lh = tf.constant([468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488])
        self.rh = tf.constant([522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542])
    
    def do_normalise_by_ref(self, xyz, ref):
            """Normalises the input array xyz by the reference array ref.

            Args:
                xyz: The input array to be normalised.
                ref: The reference array.

            Returns:
                The normalised array.
            """
            K = xyz.shape[-1]
            xyz_flat = tf.reshape(ref, [-1, K])

            m = tf.experimental.numpy.nanmean(xyz_flat, axis=0, keepdims=True)
        
            denom = tf.experimental.numpy.sum(~tf.experimental.numpy.isnan(xyz_flat),dtype=tf.float32,axis=0,keepdims=True)
            std1 = tf.experimental.numpy.nansum(tf.experimental.numpy.square(xyz_flat - m),axis=0,keepdims=True) #, dtype=tf.float32)
            s = tf.experimental.numpy.sqrt( std1 / denom)
           
            xyz = xyz - m
            xyz = xyz / s
            return xyz


    def call(self, xyz):
        
        L = len(xyz)
        if len(xyz) > self.max_length:
            # xyz = xyz[:self.max_length] #first
            # xyz = xyz[-self.max_length:] #last
            i = (L-self.max_length)//2
            xyz = xyz[i:i + self.max_length]  # center

        L = len(xyz)

        REF = tf.concat([self.lip,self.spose, self.lh, self.rh], axis=0)
        ref = tf.gather(xyz, REF, axis=1)
        xyz = self.do_normalise_by_ref(xyz,ref)

        lhand = xyz[:, self.lhand[0]:self.lhand[1],:2]
        rhand = xyz[:, self.rhand[0]:self.rhand[1],:2]
        ld = tf.reshape(lhand,(-1,21,1,2))-tf.reshape(lhand,(-1,1,21,2))
        ld = tf.math.sqrt(tf.reduce_sum((ld ** 2),-1))
        ld = tf.reshape(ld,(L, -1))
        ld = tf.gather(ld, self.triu_index, axis=1)

        rd = tf.reshape(rhand,(-1,21,1,2))-tf.reshape(rhand,(-1,1,21,2))
        rd = tf.math.sqrt(tf.reduce_sum((rd ** 2),-1))
        rd = tf.reshape(rd,(L, -1))
        rd = tf.gather(rd, self.triu_index, axis=1)

        xyz = tf.concat([
            xyz[:, self.lhand[0]:self.lhand[1]],
            xyz[:, self.rhand[0]:self.rhand[1]],
            tf.gather(xyz, self.lip, axis=1),
            tf.gather(xyz, self.spose, axis=1),
        ],1)
        dxyz = tf.pad(xyz[:-1] - xyz[1:], [[0, 1], [0, 0], [0, 0]], mode="CONSTANT")
        
        # Perform calculations
        a = xyz[1:, :, 0] - xyz[:-1, :, 0]
        b = xyz[1:, :, 1] - xyz[:-1, :, 1]
        axyz = tf.sqrt(tf.square(a) + tf.square(b))
        axyz = tf.pad(axyz, paddings=[[0, 1], [0, 0]])
        
        x = tf.concat([
            tf.reshape(xyz,(L,-1)),
            tf.reshape(dxyz,(L,-1)),
            tf.reshape(axyz,(L,-1)),
            tf.reshape(rd,(L,-1)),
            tf.reshape(ld,(L,-1)),
        ], -1)
        x = tf.where(tf.math.is_finite(x), x, tf.zeros_like(x))
        x = tf.reshape(x, (-1, num_point))

        return  x

#https://stackoverflow.com/questions/59142040/tensorflow-2-0-how-to-change-the-output-signature-while-using-tf-saved-model
class TFModel(tf.Module):
    def __init__(self,):
        super(TFModel, self).__init__()
        self.input_net = InputNet()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        outputs = self.input_net(inputs)
        return outputs

In [6]:
def run_convert_input_net_tf():
    tf_model = TFModel()
    xyz = np.random.rand(15,543,3).astype(np.float32)
    x   = tf_model(xyz)

    #----
    tf_model = TFModel()
    tf.saved_model.save(tf_model, input_net_k_tf_file, signatures={
        'serving_default': tf_model.__call__,}) #name='inputs'


def run_check_input_net():
    input_net = InputNet()
    xyz = np.random.rand(15,543,3)
    x=input_net(xyz)
    return x


def run_check_input_net_tflite():
    converter = tf.lite.TFLiteConverter.from_saved_model(input_net_k_tf_file)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    with open(input_net_k_tflite_file, 'wb') as f:
        f.write(tflite_model)
    print('converter.converter() passed !!')
    
    
#  main #################################################################
if __name__ == '__main__':
    x = run_check_input_net() ## x.shape = TensorShape([3, 82, 3])
    run_convert_input_net_tf()
    run_check_input_net_tflite()

converter.converter() passed !!


In [7]:
x.shape

TensorShape([15, 1050])

### Convert Pytorch Model -> Onnx -> TF Lite

In [8]:
class MyMultiHeadAttention(nn.Module):
    def __init__(self,
            embed_dim,
            out_dim,
            qk_dim,
            v_dim,
            num_head,
        ):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_head  = num_head
        self.qk_dim = qk_dim
        self.v_dim  = v_dim

        self.q = nn.Linear(embed_dim, qk_dim*num_head)
        self.k = nn.Linear(embed_dim, qk_dim*num_head)
        self.v = nn.Linear(embed_dim, v_dim*num_head)

        self.out = nn.Linear(v_dim*num_head, out_dim)
        self.scale = 1/(qk_dim**0.5)

    #https://github.com/pytorch/pytorch/issues/40497
    def forward(self, x):
        B,L = 1,x.shape[0]
        num_head = self.num_head
        qk_dim = self.qk_dim
        v_dim = self.v_dim

        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        q = q.reshape(L, num_head, qk_dim).permute(1,0,2).contiguous()
        k = k.reshape(L, num_head, qk_dim).permute(1,2,0).contiguous()
        v = v.reshape(L, num_head, v_dim ).permute(1,0,2).contiguous()

        dot = torch.matmul(q, k) *self.scale  # H L L
        attn = F.softmax(dot, -1)    # L L
        out1 = torch.matmul(attn, v)  
        out1 = out1.permute(1,0,2).reshape(L, v_dim*num_head).contiguous()
        out1 = self.out(out1)
        return out1

# remove mask
class TransformerBlock(nn.Module):
    def __init__(self,
        embed_dim,
        num_head,
        out_dim,
        batch_first=True,
    ):
        super().__init__()
#         self.attn  = MultiHeadAttention(embed_dim, num_head,batch_first)
        self.attn  = MyMultiHeadAttention(
            embed_dim=embed_dim,
            out_dim=embed_dim,
            qk_dim=embed_dim // num_head,
            v_dim=embed_dim // num_head,
            num_head=num_head,

        )
        self.ffn   = FeedForward(embed_dim, out_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(out_dim)

    def forward(self, x):             
        x = x + self.attn((self.norm1(x))) #[:1]
        x = x + self.ffn((self.norm2(x)))
        return x
    

class XEmbed(nn.Module):
    def __init__(self,
    ):
        super().__init__()
        self.v = nn.Sequential(
            nn.Linear(num_point, embed_dim*2, bias=True),
            nn.LayerNorm(embed_dim*2),
            nn.ReLU(inplace=True),
            nn.Linear(embed_dim*2, embed_dim, bias=True),
            nn.LayerNorm(embed_dim),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        x = self.v(x)
        return x

    
class SingleNet(nn.Module):

    def __init__(self, num_class=num_class):
        super().__init__()
        self.num_block = 1
        self.embed_dim = 384
        self.num_head  = 8
        self.max_length = max_length
        self.num_point = num_point

        pos_embed = positional_encoding(max_length, 16) # self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)
        # Linear layer to map pos_enc_dim to emb_dim
        self.pos_enc_linear = nn.Linear(16, embed_dim)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed=XEmbed()

        self.encoder = nn.ModuleList([
            TransformerBlock(
                self.embed_dim,
                self.num_head,
                self.embed_dim,
                batch_first=False
            ) for i in range(self.num_block)
        ])
        
        self.seq_layer = nn.Sequential(
            nn.Linear(embed_dim*2, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.ReLU()
        )
        self.class_layer = nn.Linear(embed_dim, num_class)
        


    def forward(self, xyz):
        x = xyz
        L = xyz.shape[0]
        x_embed = self.x_embed(xyz.flatten(1)) 
        x = x_embed[:L] + self.pos_enc_linear(self.pos_embed[:L]) #self.pos_embed[:L]
        x = torch.cat([
            self.cls_embed,
            x
        ],0)
        
        for block in self.encoder:
            x = block(x) #,x_mask)
        x = F.dropout(x,p=0.4,training=self.training)
        
        last = torch.mean(x,0).unsqueeze(0)
        
        x = torch.cat([x[[0]],last],1)
        
        x = self.seq_layer(x)
        x = F.dropout(x,p=0.3, training=self.training)
        logit = self.class_layer(x)

        return logit
    

In [9]:
single_net = SingleNet()
single_net.eval()
x = torch.rand((12,1050))
single_net(x).shape

torch.Size([1, 250])

In [10]:
tf_file = f'{convert_dir}/tf'
tflite_file = f'{convert_dir}/{name}.tflite'

def run_convert_tf(single_onnx_file, single_tf_file):
    tf_rep = prepare(onnx.load(single_onnx_file))
    tf_rep.export_graph(single_tf_file)
    print('tf_rep.export_graph() passed !!')
    
def run_convert_onnx(single_net, single_onnx_file):
    single_tensor = torch.zeros(max_length,num_point)
    torch.onnx.export(
        single_net,                   # model being run
        single_tensor,                # model input (or a tuple for multiple inputs)
        single_onnx_file,             # where to save the model (can be a file or file-like object)
        export_params = True,         # store the trained parameter weights inside the model file
        opset_version = 12,#12,       # the ONNX version to export the model to
        do_constant_folding=True,     # whether to execute constant folding for optimization
        input_names =  ['inputs'],    # the model's input names
        output_names = ['outputs'],   # the model's output names
        dynamic_axes={
            'inputs': {0: 'length'},
        },
        #verbose = True,
    )
    print('torch.onnx.export() passed !!')
    
    for f in [single_onnx_file]:
        if f is None: continue
        model = onnx.load(f)
        onnx.checker.check_model(model)
        model_simple, check = onnxsim.simplify(model)
        onnx.save(model_simple, f)
    print('onnx simplify() passed !!')
    

for fold, cfile in enumerate(checkpoint_files):
    single_net = SingleNet()
    if cfile is not None:
        f = torch.load(cfile, map_location=lambda storage, loc: storage)
        state_dict = f['model']
        state_dict['pos_embed'] = state_dict['pos_embed'][:max_length]
        print(single_net.load_state_dict(state_dict, strict=False))  # True  False

    single_net.eval();
    
    single_onnx_file = f'{convert_dir}/single_net_fold_{fold}_onnx'
    input_tf_file     = f'input_tf_fold_{fold}'
    single_tf_file    = f'{convert_dir}/single_tf_fold_{fold}'
    
    run_convert_onnx(single_net, single_onnx_file)
    
    run_convert_tf(single_onnx_file, single_tf_file)
    

<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!
<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!
<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!
<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!
<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!
<All keys matched successfully>
torch.onnx.export() passed !!
onnx simplify() passed !!
tf_rep.export_graph() passed !!


In [11]:
!ls -al convert_dir

total 56384
drwxr-xr-x 9 root root    4096 Apr 30 09:19 .
drwxr-xr-x 3 root root    4096 Apr 30 09:18 ..
drwxr-xr-x 4 root root    4096 Apr 30 09:18 input_net.fold_all.k.tf
-rw-r--r-- 1 root root   18172 Apr 30 09:18 input_net.fold_all.k.tflite
-rw-r--r-- 1 root root 9609672 Apr 30 09:18 single_net_fold_0_onnx
-rw-r--r-- 1 root root 9609672 Apr 30 09:18 single_net_fold_1_onnx
-rw-r--r-- 1 root root 9609672 Apr 30 09:18 single_net_fold_2_onnx
-rw-r--r-- 1 root root 9609672 Apr 30 09:18 single_net_fold_3_onnx
-rw-r--r-- 1 root root 9609672 Apr 30 09:19 single_net_fold_4_onnx
-rw-r--r-- 1 root root 9609672 Apr 30 09:19 single_net_fold_5_onnx
drwxr-xr-x 4 root root    4096 Apr 30 09:18 single_tf_fold_0
drwxr-xr-x 4 root root    4096 Apr 30 09:18 single_tf_fold_1
drwxr-xr-x 4 root root    4096 Apr 30 09:18 single_tf_fold_2
drwxr-xr-x 4 root root    4096 Apr 30 09:19 single_tf_fold_3
drwxr-xr-x 4 root root    4096 Apr 30 09:19 single_tf_fold_4
drwxr-xr-x 4 root root    4096 Apr 30 09:19 sing

#### Convert to TF Lite

In [12]:
def run_convert_tflite():
    single_net_tf_files = [f'{convert_dir}/single_tf_fold_{fold}' for fold in range(len(checkpoint_files))]
    
    class TFModel(tf.Module):
        def __init__(self):
            super(TFModel, self).__init__()
            self.input_net  = tf.saved_model.load(input_net_k_tf_file)
            
            self.single_net0 = tf.saved_model.load(single_net_tf_files[0])
            self.single_net1 = tf.saved_model.load(single_net_tf_files[1])
            self.single_net2 = tf.saved_model.load(single_net_tf_files[2])
            self.single_net3 = tf.saved_model.load(single_net_tf_files[3])
            self.single_net4 = tf.saved_model.load(single_net_tf_files[4])
            self.single_net5 = tf.saved_model.load(single_net_tf_files[5])
            
            self.input_net.trainable = False
            self.single_net0.trainable = False
            self.single_net1.trainable = False
            self.single_net2.trainable = False
            self.single_net3.trainable = False
            self.single_net4.trainable = False
            self.single_net5.trainable = False
            
        @tf.function(input_signature=[
            tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
        ])
        def call(self, inputs):
            y = {}
            
            x  = self.input_net(inputs)
            
            y0 = self.single_net0(inputs=x)['outputs']
            y1 = self.single_net1(inputs=x)['outputs']
            y2 = self.single_net2(inputs=x)['outputs']
            y3 = self.single_net3(inputs=x)['outputs']
            y4 = self.single_net4(inputs=x)['outputs']
            y5 = self.single_net5(inputs=x)['outputs']
            
            outputs = (y0 + y1 + y2 + y3 + y4 + y5 )/6.
#             outputs = (y0 + y1 + y2 + y3 + y4 )/5.
#             outputs = (y0 + y1)/2.0
#             outputs = y0
#             outputs = (y0 + y1 + y2)/3.0
#             outputs = (y0 + y1 + y3 + y2 )/4.
            return {'outputs': outputs }
    
    
    tfmodel = TFModel()
    tf.saved_model.save(tfmodel, tf_file, signatures={'serving_default': tfmodel.call})
    print('tf.saved_model() passed !!')

    converter = tf.lite.TFLiteConverter.from_saved_model(tf_file)
    # converter.target_spec.supported_ops = [
    #     tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    #     tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
    # ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]

    #converter.allow_custom_ops = True
    #converter.experimental_new_converter = True
    tf_lite_model = converter.convert()
    with open(tflite_file, 'wb') as f:
        f.write(tf_lite_model)
    print('tflite convert() passed !!')
    
        
run_convert_tflite()

tf.saved_model() passed !!
tflite convert() passed !!


In [13]:
if 1: ##debug
    DF_INDEX = [
    180     ,#train_landmark_files/4718/1007273104.parquet,4718,1007273104,white,3
    1       ,#train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,11
    81543   ,#train_landmark_files/2044/4693753.parquet,2044,4693753,orange,15
    0       ,#train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,23
    2       ,#train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,105
    13      ,#train_landmark_files/26734/1000661926.parquet,26734,1000661926,mitten,141
    4622    ,#train_landmark_files/28656/1192107487.parquet,28656,1192107487,child,154
    45      ,#train_landmark_files/26734/1001931356.parquet,26734,1001931356,cloud,225
    ]
    sign_to_label = json.load(open("/kaggle/input/asl-signs/sign_to_prediction_index_map.json", "r"))
    kaggle_df = pd.read_csv('/kaggle/input/asl-folds/asl_train_5folds_sgkf.csv') #f'{root_dir}/data/asl-signs/train.ver01.csv')
    kaggle_df.loc[:, 'label'] = kaggle_df.sign.map(sign_to_label)


    interpreter = tflite.Interpreter(tflite_file)

    #---
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_shape  = input_details[0]['shape']
    output_shape = output_details[0]['shape']
    print('')
    print('input_details :', input_details)
    print('input_shape   :', input_shape)
    print('output_shape  :', output_shape)
    print('')

    #---

    found_signatures = list(interpreter.get_signature_list().keys())
    print('found_signatures :', found_signatures)

    # if REQUIRED_SIGNATURE not in found_signatures:
    # 	raise KernelEvalException('Required input signature not found.')

    prediction_fn = interpreter.get_signature_runner("serving_default")

    ##todo lite api to print tflite model spec
    print('')
    print(' *** debug tflite runtime ***')
    for i in DF_INDEX: # * 10000:
        d = kaggle_df.iloc[i]
        pq_file = f'/kaggle/input/asl-signs/{d.path}'
        xyz = load_relevant_data_subset(pq_file)
        output = prediction_fn(inputs=xyz)#[:98]

        y = output['outputs']
        xyz_flat = xyz.reshape(-1)
        y_flat = y.reshape(-1)

        # print(d)
        print('------------------------------')
        print('xyz  :', xyz.shape)
        print('y    :', y.shape)
        print('xyz NaN   :', np.isnan(xyz_flat).sum())
        print('xyz values:', xyz_flat[:5])
        print('y   values:', y_flat[:5])
        print('y   top5  :', np.argsort(-y_flat)[:5])
        print('truth     :', d.label, d.sign)
        print('')


input_details : [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([  1, 543,   3], dtype=int32), 'shape_signature': array([ -1, 543,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
input_shape   : [  1 543   3]
output_shape  : [  1 250]

found_signatures : ['serving_default']

 *** debug tflite runtime ***
------------------------------
xyz  : (3, 543, 3)
y    : (1, 250)
xyz NaN   : 189
xyz values: [ 0.5168196   0.482137   -0.04360763  0.5173948   0.43922532]
y   values: [-0.3382505  -0.6208968  -0.33524254 -0.08385886 -0.34519818]
y   top5  : [238 248 132 195 166]
truth     : 238 white

------------------------------
xyz  : (11, 543, 3)
y    : (1, 250)
xyz NaN   : 1260
xyz values: [ 0.57777625  0.5093604  -0.05024542  0.572079    0.47016424]
y   values: [-0.52848387 -0.49910167 -0.5

In [14]:
# tflite_file = '/kaggle/working/exp3.tflite'
# '/kaggle/input/asl-demo/run20-aug3-xyz2.tflite'

mode = 'submit' #debug #submit

import pandas as pd
import numpy as np
import os
import shutil
from datetime import datetime
from timeit import default_timer as timer

print('import ok')
'''
Your model must also require less than 40 MB in memory and 
perform inference with less than 100 milliseconds of latency per video. 
Expect to see approximately 40,000 videos in the test set. 
We allow an additional 10 minute buffer for loading the data and miscellaneous overhead.

'''
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError

shutil.copyfile(tflite_file, 'model.tflite') 
!zip submission.zip  'model.tflite'
!ls -l

print('tflite_file:', tflite_file)
print(f'submit ok')

import ok
  adding: model.tflite (deflated 8%)
total 54288
----------  1 root root    53613 Apr 30 09:21 __notebook__.ipynb
drwxr-xr-x 10 root root     4096 Apr 30 09:21 convert_dir
-rw-r--r--  1 root root 28928612 Apr 30 09:21 model.tflite
-rw-r--r--  1 root root 26595980 Apr 30 09:21 submission.zip
tflite_file: convert_dir/exp81.tflite
submit ok


In [15]:
# !rm -rf convert_dir